## Acessando o banco de dados ChEMBL pelo ChEMBL webresource client

Para usar essa ferramenta, é necessário instalar o cliente utilizando:

`pip install chembl_webresource_client`

Tendo instalado o cliente, é necessário importá-lo, usando:

In [1]:
from chembl_webresource_client.new_client import new_client

O [repositório no GitHub](https://github.com/chembl/chembl_webresource_client) sugere algumas formas de uso, como:

### Procurar uma estrutura (molecule) pelo nome

In [2]:
molecule = new_client.molecule
res = molecule.search('viagra')
type(res)

chembl_webresource_client.query_set.QuerySet

Veja que o resultado é um objeto do tipo QuerySet, contendo um extenso dicionário:

In [3]:
res

[{'atc_classifications': ['G04BE03'], 'availability_type': 1, 'biotherapeutic': None, 'black_box_warning': 0, 'chebi_par_id': 9139, 'chirality': 2, 'cross_references': [{'xref_id': 'sildenafil%20citrate', 'xref_name': 'sildenafil citrate', 'xref_src': 'DailyMed'}, {'xref_id': '26748898', 'xref_name': 'SID: 26748898', 'xref_src': 'PubChem'}, {'xref_id': '50085897', 'xref_name': 'SID: 50085897', 'xref_src': 'PubChem'}], 'dosed_ingredient': True, 'first_approval': 1998, 'first_in_class': 0, 'helm_notation': None, 'indication_class': 'Impotence Therapy', 'inorganic_flag': 0, 'max_phase': 4, 'molecule_chembl_id': 'CHEMBL192', 'molecule_hierarchy': {'molecule_chembl_id': 'CHEMBL192', 'parent_chembl_id': 'CHEMBL192'}, 'molecule_properties': {'alogp': '1.61', 'aromatic_rings': 3, 'cx_logd': '1.16', 'cx_logp': '1.23', 'cx_most_apka': '7.63', 'cx_most_bpka': '5.98', 'full_molformula': 'C22H30N6O4S', 'full_mwt': '474.59', 'hba': 8, 'hba_lipinski': 10, 'hbd': 1, 'hbd_lipinski': 1, 'heavy_atoms': 3

In [4]:
# Chaves do dicionário
res[0].keys()

dict_keys(['atc_classifications', 'availability_type', 'biotherapeutic', 'black_box_warning', 'chebi_par_id', 'chirality', 'cross_references', 'dosed_ingredient', 'first_approval', 'first_in_class', 'helm_notation', 'indication_class', 'inorganic_flag', 'max_phase', 'molecule_chembl_id', 'molecule_hierarchy', 'molecule_properties', 'molecule_structures', 'molecule_synonyms', 'molecule_type', 'natural_product', 'oral', 'parenteral', 'polymer_flag', 'pref_name', 'prodrug', 'score', 'structure_type', 'therapeutic_flag', 'topical', 'usan_stem', 'usan_stem_definition', 'usan_substem', 'usan_year', 'withdrawn_class', 'withdrawn_country', 'withdrawn_flag', 'withdrawn_reason', 'withdrawn_year'])

In [5]:
print("Tipo de molécula:", res[0]['molecule_type'])
print("Número identificador no ChEMBL:", res[0]['molecule_chembl_id'])
print("É um medicamento tomado por via oral?", res[0]['oral'])
print("SMILES:", res[0]['molecule_structures']['canonical_smiles'])  # molecule_structures é um dict dentro de um dict

Tipo de molécula: Small molecule
Número identificador no ChEMBL: CHEMBL192
É um medicamento tomado por via oral? True
SMILES: CCCc1nn(C)c2c(=O)[nH]c(-c3cc(S(=O)(=O)N4CCN(C)CC4)ccc3OCC)nc12


### Procurar um alvo (target) pelo nome do gene

In [6]:
target = new_client.target
gene_name = 'BRD4'
res = target.search(gene_name)

In [7]:
res[0].keys()

dict_keys(['cross_references', 'organism', 'pref_name', 'score', 'species_group_flag', 'target_chembl_id', 'target_components', 'target_type', 'tax_id'])

### Encontre compostos similares à aspirina (similaridade > 70%)

In [8]:
molecule = new_client.molecule
similarity = new_client.similarity
aspirin_chembl_id = molecule.search('aspirin')[0]['molecule_chembl_id']
res = similarity.filter(chembl_id=aspirin_chembl_id, similarity=70)

In [9]:
for item in res:
    print(item['molecule_structures']['canonical_smiles'])

CC(=O)Oc1ccccc1C(=O)O.NCCCC[C@H](N)C(=O)O
CC(=O)Oc1ccccc1C(=O)O.NCCCCC(N)C(=O)O
CC(=O)Oc1ccccc1C(=O)[O-].CC(=O)Oc1ccccc1C(=O)[O-].NC(N)=O.[Ca+2]
CC(=O)Oc1ccccc1C(=O)O.CC(=O)Oc1ccccc1C(=O)O.NC(N)=O
CC(=O)Oc1ccccc1C(=O)Oc1ccccc1C(=O)O
O=C(O)Oc1ccccc1C(=O)O
CC(=O)Oc1cccc(C(=O)O)c1OC(C)=O


### Gerar um dataframe com todos os inibidores de um alvo

Escrevi uma função que recebe como entrada o código ChEMBL do alvo desejado. Para descobrir o código de um alvo, acesse o [site do ChEMBL](https://www.ebi.ac.uk/chembl/) e procure pelo alvo na barra *Search in ChEMBL*. A função retorna um dataframe contendo as estruturas (SMILES) e atividades (pChEMBL) dos compostos

In [10]:
import pandas as pd

In [11]:
activities = new_client.activity

In [12]:
def get_inhibitors(target):
    """
    Returns a dataframe with SMILES and pChEMBL values of inhibitors for a given target
    Example: target = Cathepsin B = "CHEMBL4072"
    """
    inhibitors = activities.filter(target_chembl_id=target, pchembl_value__isnull=False)
    data = [[item["canonical_smiles"], item["pchembl_value"]] for item in inhibitors]
    df = pd.DataFrame(data, columns=["SMILES", "pChEMBL"])
    return df

In [13]:
# Encontrando inibidores da enzima Catepsina B
df = get_inhibitors("CHEMBL4072")  # código ChEMBL da Catepsina B
df.dropna(inplace=True)
df

,SMILES,pChEMBL
0,CC(C)C[C@H](C=O)NC(=O)[C@@H](NS(=O)(=O)c1ccc(F...,7.11
1,O=C(NC(=O)[C@H](Cc1cc(I)c(O)c(I)c1)NC(=O)[C@H]...,6.47
2,CC(C)C[C@H](NC(=O)OCc1ccccc1)C(=O)N[C@@H](CC(C...,8.21
3,CC(C)[C@H](NC(=O)OCc1ccccc1)C(=O)N[C@@H]1C(=O)...,4.52
4,O=C(N[C@@H](Cc1ccccc1)C(=O)N[C@@H]1C(=O)N2CCO[...,5.75
...,...,...
1358,C=CC(=O)Nc1cc(Oc2nc(Nc3ccc(N4CCN(C)CC4)cc3OC)c...,5.52
1359,C[C@H](NC(=O)[C@H](Cc1ccccc1)NC(=O)OCc1ccccc1)...,6.52
1360,O=C(CF)CNC(=O)[C@H](Cc1ccccc1)NC(=O)c1ccccc1,6.16
1361,O=C(CF)CNC(=O)[C@H](Cc1ccccc1)NC(=O)c1cccc2ccc...,6.70
